In [44]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = set(stopwords.words('english'))

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [45]:
# create documents
depression_set = []
for i in xrange(781):
    f = open('depress_corpus/corpus_'+str(i))
    line = ""
    for j in f:
        line += j.decode('utf-8')
    depression_set.append(line)
    f.close()
control_set = []
for i in xrange(824):
    f = open('other_corpus/corpus_'+str(i))
    line = ""
    for j in f:
        line += j.decode('utf-8')
    control_set.append(line)
    f.close()
doc_set = depression_set + control_set
label = [1]*781 + [0]*824

In [46]:
# list for tokenized documents in loop
texts = []
# loop through document list
for i in doc_set:
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop and len(i)!=1]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stemmed_tokens)
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [47]:
from scipy.sparse import csr_matrix
data = []
row = []
col = []
for (N,c) in enumerate(corpus):
    for i in c:
        data.append(i[1])
        row.append(N)
        col.append(i[0])
BoW_feature = csr_matrix((data,(row,col))).toarray()

In [48]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary)

In [62]:
doc_lda = ldamodel[corpus]
data = []
row = []
col = []
for (N,doc) in enumerate(doc_lda):
    for i in doc:
        data.append(i[1])
        row.append(N)
        col.append(i[0])
LDA_feature = csr_matrix((data,(row,col))).toarray()

In [63]:
print BoW_feature.shape
print LDA_feature.shape

(1605, 9453)
(1605, 20)


In [ ]:
numpy.save('bow_feature.npy',BoW_feature)
numpy.save('lda_feature.npy',BoW_feature)

In [52]:
print(ldamodel.print_topics(num_topics=20, num_words=10))

[(0, u'0.014*"go" + 0.008*"one" + 0.006*"time" + 0.006*"get" + 0.005*"know" + 0.005*"like" + 0.004*"could" + 0.004*"work" + 0.004*"think" + 0.004*"would"'), (1, u'0.015*"feel" + 0.010*"like" + 0.007*"would" + 0.007*"go" + 0.006*"time" + 0.006*"get" + 0.005*"know" + 0.005*"one" + 0.005*"someth" + 0.005*"need"'), (2, u'0.011*"like" + 0.011*"get" + 0.006*"year" + 0.006*"back" + 0.006*"feel" + 0.006*"time" + 0.006*"day" + 0.006*"tri" + 0.005*"would" + 0.005*"want"'), (3, u'0.008*"go" + 0.008*"time" + 0.007*"one" + 0.006*"get" + 0.006*"like" + 0.005*"say" + 0.005*"could" + 0.005*"feel" + 0.005*"help" + 0.005*"year"'), (4, u'0.009*"go" + 0.008*"like" + 0.008*"would" + 0.007*"back" + 0.006*"look" + 0.006*"know" + 0.006*"get" + 0.006*"year" + 0.006*"make" + 0.005*"want"'), (5, u'0.010*"go" + 0.009*"peopl" + 0.009*"want" + 0.008*"would" + 0.008*"get" + 0.008*"feel" + 0.007*"time" + 0.007*"year" + 0.007*"like" + 0.006*"depress"'), (6, u'0.013*"get" + 0.011*"feel" + 0.010*"go" + 0.010*"time" + 0.

[(0, 0.010000000068013774),
 (1, 0.010000000086333411),
 (2, 0.010000000132431021),
 (3, 0.010000000167327524),
 (4, 0.010000000211210242),
 (5, 0.01000000012438532),
 (6, 0.010000000180284314),
 (7, 0.010000000165587277),
 (8, 0.010000000173138613),
 (9, 0.010000000131704417),
 (10, 0.010000000159253305),
 (11, 0.010000000126559732),
 (12, 0.010000000121553594),
 (13, 0.010000000208291313),
 (14, 0.010000000146996247),
 (15, 0.010000000068586274),
 (16, 0.010000000522428148),
 (17, 0.010000000162195013),
 (18, 0.80999999685326807),
 (19, 0.010000000190452376)]